In [4]:
from flow import FlowProject
from project import MartiniProject

In [12]:
jobs = MartiniProject().find_jobs()

In [13]:
for job in jobs:
    print(job)

591acce991221b1138cfe5343cfb1a44
83997b1e23c22aa720eecabfc3265ec5
01a63b22db35a5f7008102d12b70f4da
85928b5c22176a2ebb352570568cf244
3cd5b537b11def653cc59df3fb37c229
f42be03154f5679bd6010a5022ad617f
22d4866ce5bd19769f025982fa117bdd
64d9244812a356e19738e0396c23b650
77e0d9ffe33c41244aadc65f75139d4e
3200e6249c43d458249916a5fe8a3614
763ecfc477b8ee1f032e78bef797fdf6


In [19]:
job = list(jobs)[0]
job

Job(project=MartiniProject('/Users/tbereau/work/research/martignac/scripts/martini_fep'), statepoint={'lambda_state': 8})

In [24]:
with open(job.fn("run_8.xvg")) as p:
    lines = p.read()

In [28]:
lines[0:100]

'# This file was created Tue Jun 20 15:04:45 2023\n# Created by:\n#                      :-) GROMACS - '